In [34]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence,RunnableBranch,RunnablePassthrough

In [35]:
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

In [36]:
prompt1 = PromptTemplate(
    template="Please generate a report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Make a summary of the following and also wite at the start that it  is a summary \n {text}",
    input_variables=["text"]    
)

parser = StrOutputParser()

In [37]:
report_gen_chain = RunnableSequence(prompt1,model,parser)

In [47]:
branch = RunnableBranch(
    (lambda x: len(x.split())>50, RunnableSequence(prompt2,model,parser)),
    RunnablePassthrough()
)

In [48]:
complete_chain = RunnableSequence(report_gen_chain,branch)

In [ ]:
res = complete_chain.invoke({'topic':'Pakistani Culture'})
print(res)